In [5]:
import tensorflow.keras
from PIL import Image, ImageOps

import numpy as np
import cv2

#from motor import *
def mark_img(img, blue_threshold=200, green_threshold=200, red_threshold=200): # 1. 흰색 차선 찾기
    mark = img
    #  BGR 제한 값 (기준)
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]

    # BGR 제한 값(기준)보다 작으면 검은색으로
    thresholds = (img[:,:,0] < bgr_threshold[0]) \
                | (img[:,:,1] < bgr_threshold[1]) \
                | (img[:,:,2] < bgr_threshold[2])
    mark[thresholds] = [0,0,0] # 검정색으로 
    return mark
 
def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # ROI 셋팅
    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)

    return ROI_image

# 허프변환 라인 그리기 (확인용)
def draw_lines(img, lines, color=[255, 255, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)\
            
# 허프 변환
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): 
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)

    return lines

# 두 이미지 operlap 하기
def weighted_img(img, initial_img, α=1, β=1., λ=0.): 
    return cv2.addWeighted(initial_img, α, img, β, λ)

# '대표선' 구하기 (62, 63줄 코드 이해 X )
def get_fitline(img, f_lines): 
    lines = np.squeeze(f_lines)
    if(len(lines.shape) == 1): #선이 하나밖에 없는 경우 배열의 모양 따로 조정
        lines = lines.reshape(2,2)
    else:
        lines = lines.reshape(lines.shape[0]*2,2)

    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]/2+70)-y)/vy*vx + x) , int(img.shape[0]/2+70)
    x2, y2 = int(((img.shape[0]/2-25)-y)/vy*vx + x) , int(img.shape[0]/2-25)
    result = [x1,y1,x2,y2]
    
    return result

# '대표선' 그리기
def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10):
        # cv2.line(이미지, 시작좌표(0, 0), 끝좌표(500, 500), 색깔, 두께)
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness) 

# '대표선' 이용해서 -> 소실점 구하기 (공식)
def expression(x1,y1,x2,y2,x3,y3,x4,y4):
    m_a = (y2 - y1) / (x2 -x1)
    m_b = (y4 - y3) / (x4 - x3)
    n_a = -((y2 - y1) / (x2 - x1) * x1 ) + y1
    n_b = -((y4 - y3) / (x4 -x3) * x3 ) + y3
    x = (n_b - n_a) / (m_a - m_b) 
    y = m_a * ((n_b - n_a) / (m_a - m_b)) + n_a 

    return x,y
 

#=======================================

 

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)


# Load the model
model = tensorflow.keras.models.load_model('keras_model.h5',compile = False)

 
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


cap = cv2.VideoCapture(0)



while(cap.isOpened()):

    ret,image = cap.read()

    image2 = cv2.resize(image, (224, 224)) # window 사이즈 조정
    

    height, width = image.shape[:2] # 이미지 높이, 너비
    
    #cv2.imshow('result2',image2)
    cv2.imshow('result2',image2) # 결과 이미지 출력
    cv2.imshow('result',image) # 결과 이미지 출력
 

    #turn the image into a numpy array

    image_array = np.asarray(image2)

    

    # Normalize the image

    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    
 

    # Load the image into the array

    data[0] = normalized_image_array

    # run the inference

    prediction = model.predict(data)

 

    direction = ''

    if prediction[0][0] >= 0.8:
        #direction = F
        print('forward!!')

 

    elif prediction[0][1] >= 0.8:
        #direction = FR
        print('right!!')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

forward!!
forward!!
forward!!
forward!!
forward!!
forward!!
